# GOAT Index: Análisis Comparativo de Michael Jordan, Kobe Bryant y LeBron James

## Introducción

El debate sobre quién es el mejor jugador de la historia de la NBA (GOAT – Greatest Of All Time) es uno de los más recurrentes y polémicos en el deporte.

Este proyecto busca construir un **índice cuantitativo (GOAT Index)** que combine:
- rendimiento ofensivo
- impacto defensivo
- eficiencia
- éxito colectivo (títulos) 
para comparar a tres de los principales candidatos al GOAT:
**Michael Jordan, Kobe Bryant y LeBron James**.

El análisis se apoya en datos oficiales de la NBA y en datasets históricos, con el objetivo de ofrecer una comparación transparente, reproducible y defendible.


In [1]:
import pandas as pd
import numpy as np

from nba_api.stats.endpoints import playercareerstats


In [2]:
players = pd.read_csv("/Users/lautarocardinisilvestri/Downloads/Ironhack/Semana 4/Proyecto NBA/NBA_PLAYERS.csv")
teams = pd.read_csv("/Users/lautarocardinisilvestri/Downloads/Ironhack/Semana 4/Proyecto NBA/NBA_TEAMS.csv")
finals = pd.read_csv("/Users/lautarocardinisilvestri/Downloads/Ironhack/Semana 4/Proyecto NBA/NBA_Finals_and_MVP.csv")

In [3]:
GOATS_IDS = {
    "Michael Jordan": 893,
    "Kobe Bryant": 977,
    "LeBron James": 2544
}


In [4]:
def get_player_career(player_name, player_id):
    career = playercareerstats.PlayerCareerStats(player_id=player_id)
    df = career.get_data_frames()[0]
    df["PLAYER_NAME"] = player_name
    return df


In [5]:
dfs = []

for name, pid in GOATS_IDS.items():
    dfs.append(get_player_career(name, pid))

career_df = pd.concat(dfs, ignore_index=True)


In [6]:
career_df[["PLAYER_NAME", "SEASON_ID", "GP", "MIN", "PTS"]].head()


,PLAYER_NAME,SEASON_ID,GP,MIN,PTS
0,Michael Jordan,1984-85,82,3144,2313
1,Michael Jordan,1985-86,18,451,408
2,Michael Jordan,1986-87,82,3281,3041
3,Michael Jordan,1987-88,82,3311,2868
4,Michael Jordan,1988-89,81,3255,2633


In [7]:
finals.columns


Index(['index', 'Year', 'Western Champion', 'Eastern Champion', 'Result',
       'NBA Champion', 'NBA Vice-Champion', 'Final Sweep ?', 'MVP Name',
       'MVP Height (m)', 'MVP Height (ft)', 'MVP Position', 'MVP Team',
       'MVP Nationality', 'MVP status'],
      dtype='object')

## Preparación de los datos

En esta sección se limpian y transforman las estadísticas originales
para obtener métricas comparables entre jugadores con diferentes
minutos y contextos de juego.

Se utilizan métricas normalizadas por minuto y por 36 minutos,
estándar común en análisis NBA.


In [8]:
cols_clean = [
    "PLAYER_NAME", "SEASON_ID", "GP", "MIN",
    "PTS", "REB", "AST", "STL", "BLK",
    "FG_PCT", "FG3_PCT", "FT_PCT"
]

GOAT_df = career_df[cols_clean].copy()
GOAT_df.head()


,PLAYER_NAME,SEASON_ID,GP,MIN,PTS,REB,AST,STL,BLK,FG_PCT,FG3_PCT,FT_PCT
0,Michael Jordan,1984-85,82,3144,2313,534,481,196,69,0.515,0.173,0.845
1,Michael Jordan,1985-86,18,451,408,64,53,37,21,0.457,0.167,0.840
2,Michael Jordan,1986-87,82,3281,3041,430,377,236,125,0.482,0.182,0.857
3,Michael Jordan,1987-88,82,3311,2868,449,485,259,131,0.535,0.132,0.841
4,Michael Jordan,1988-89,81,3255,2633,652,650,234,65,0.538,0.276,0.850


In [9]:
for stat in ["PTS", "REB", "AST", "STL", "BLK"]:
    GOAT_df[f"{stat}_PER_MIN"] = GOAT_df[stat] / GOAT_df["MIN"]

GOAT_df[
    ["PLAYER_NAME", "SEASON_ID", "MIN",
     "PTS_PER_MIN", "REB_PER_MIN", "AST_PER_MIN"]
].head()


,PLAYER_NAME,SEASON_ID,MIN,PTS_PER_MIN,REB_PER_MIN,AST_PER_MIN
0,Michael Jordan,1984-85,3144,0.735687,0.169847,0.152990
1,Michael Jordan,1985-86,451,0.904656,0.141907,0.117517
2,Michael Jordan,1986-87,3281,0.926852,0.131058,0.114904
3,Michael Jordan,1987-88,3311,0.866204,0.135609,0.146481
4,Michael Jordan,1988-89,3255,0.808909,0.200307,0.199693


In [10]:
for stat in ["PTS", "REB", "AST", "STL", "BLK"]:
    GOAT_df[f"{stat}_PER_36"] = GOAT_df[f"{stat}_PER_MIN"] * 36

GOAT_df[
    ["PLAYER_NAME", "SEASON_ID",
     "PTS_PER_36", "REB_PER_36", "AST_PER_36",
     "STL_PER_36", "BLK_PER_36"]
].head()


,PLAYER_NAME,SEASON_ID,PTS_PER_36,REB_PER_36,AST_PER_36,STL_PER_36,BLK_PER_36
0,Michael Jordan,1984-85,26.484733,6.114504,5.507634,2.244275,0.790076
1,Michael Jordan,1985-86,32.567627,5.108647,4.230599,2.953437,1.676275
2,Michael Jordan,1986-87,33.366657,4.718074,4.136544,2.589454,1.371533
3,Michael Jordan,1987-88,31.183328,4.881909,5.273331,2.816068,1.424343
4,Michael Jordan,1988-89,29.120737,7.211060,7.188940,2.588018,0.718894


In [11]:
metrics_GOAT = GOAT_df[
    [
        "PLAYER_NAME", "SEASON_ID",
        "PTS_PER_36", "REB_PER_36", "AST_PER_36",
        "STL_PER_36", "BLK_PER_36",
        "FG_PCT", "FG3_PCT", "FT_PCT"
    ]
].copy()

metrics_GOAT.head()


,PLAYER_NAME,SEASON_ID,PTS_PER_36,REB_PER_36,AST_PER_36,STL_PER_36,BLK_PER_36,FG_PCT,FG3_PCT,FT_PCT
0,Michael Jordan,1984-85,26.484733,6.114504,5.507634,2.244275,0.790076,0.515,0.173,0.845
1,Michael Jordan,1985-86,32.567627,5.108647,4.230599,2.953437,1.676275,0.457,0.167,0.840
2,Michael Jordan,1986-87,33.366657,4.718074,4.136544,2.589454,1.371533,0.482,0.182,0.857
3,Michael Jordan,1987-88,31.183328,4.881909,5.273331,2.816068,1.424343,0.535,0.132,0.841
4,Michael Jordan,1988-89,29.120737,7.211060,7.188940,2.588018,0.718894,0.538,0.276,0.850


DEFINIMOS LA TRAYECTORIA DE LOS TRES JUGADORES PARA CONOCER LOS EQUIPOS DONDE JUGARON Y SI, EN SU CARRERA, OBTUVIERON TÍTULOS

In [12]:
player_teams = {
    "Michael Jordan": ["Chicago Bulls"],
    "Kobe Bryant": ["Los Angeles Lakers"],
    "LeBron James": ["Miami Heat", "Cleveland Cavaliers", "Los Angeles Lakers"]
}


In [13]:
championships_df = pd.DataFrame({
    "PLAYER_NAME": ["Michael Jordan", "Kobe Bryant", "LeBron James"],
    "CHAMPIONSHIPS": [6, 5, 4]
})


In [14]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()

In [15]:
goat_scaled = metrics_GOAT.copy()

cols_to_scale = [
    "PTS_PER_36",
    "REB_PER_36",
    "AST_PER_36",
    "STL_PER_36",
    "BLK_PER_36",
    "FG_PCT",
    "FG3_PCT",
    "FT_PCT"
]

goat_scaled[cols_to_scale] = scaler.fit_transform(
    goat_scaled[cols_to_scale]
)


In [16]:
weights = {
    "PTS_PER_36": 0.25,
    "REB_PER_36": 0.15,
    "AST_PER_36": 0.15,
    "STL_PER_36": 0.05,
    "BLK_PER_36": 0.05,
    "FG_PCT": 0.15,
    "FG3_PCT": 0.10,
    "FT_PCT": 0.10
}

goat_scaled["GOAT_INDEX"] = sum(
    goat_scaled[m] * w for m, w in weights.items()
)


In [17]:
goat_final = goat_scaled[["PLAYER_NAME", "GOAT_INDEX"]].copy()


NORMALIZAMOS Y PONDERAMOS LA VARIABLE "CAMPEONATOS" PARA INCLUIRLA EN NUESTRO ANÁLISIS

In [18]:
championships_df["CHAMPIONSHIPS_NORM"] = (
    championships_df["CHAMPIONSHIPS"] /
    championships_df["CHAMPIONSHIPS"].max()
)


INTEGRAMOS LOS DOS DF A FINES QUE QUEDEN UNIDOS Y CONTRIBUYAN A SU ANÁLISIS FINAL

In [19]:
goat_final = goat_final.merge(
    championships_df[["PLAYER_NAME", "CHAMPIONSHIPS_NORM"]],
    on="PLAYER_NAME",
    how="left"
)


In [20]:
goat_final


,PLAYER_NAME,GOAT_INDEX,CHAMPIONSHIPS_NORM
0,Michael Jordan,0.525108,1.000000
1,Michael Jordan,0.558956,1.000000
2,Michael Jordan,0.567350,1.000000
3,Michael Jordan,0.585963,1.000000
4,Michael Jordan,0.688427,1.000000
5,Michael Jordan,0.674839,1.000000
6,Michael Jordan,0.649937,1.000000
7,Michael Jordan,0.565787,1.000000
8,Michael Jordan,0.603250,1.000000
9,Michael Jordan,0.464751,1.000000


In [21]:
goat_final["GOAT_INDEX_FINAL"] = (
    goat_final["GOAT_INDEX"] * 0.9 +
    goat_final["CHAMPIONSHIPS_NORM"] * 0.1
)


In [22]:
goat_final = goat_final.sort_values(
    "GOAT_INDEX_FINAL",
    ascending=False
).reset_index(drop=True)


In [23]:
goat_final[[
    "PLAYER_NAME",
    "GOAT_INDEX_FINAL",
    "GOAT_INDEX",
    "CHAMPIONSHIPS_NORM"
]]


,PLAYER_NAME,GOAT_INDEX_FINAL,GOAT_INDEX,CHAMPIONSHIPS_NORM
0,Michael Jordan,0.719585,0.688427,1.000000
1,Michael Jordan,0.707355,0.674839,1.000000
2,Michael Jordan,0.684943,0.649937,1.000000
3,LeBron James,0.670794,0.671252,0.666667
4,Michael Jordan,0.642925,0.603250,1.000000
5,LeBron James,0.639066,0.635999,0.666667
6,LeBron James,0.632457,0.628656,0.666667
7,LeBron James,0.631752,0.627872,0.666667
8,LeBron James,0.630088,0.626023,0.666667
9,Michael Jordan,0.627367,0.585963,1.000000


In [24]:
ranking_final = goat_final[[
    "PLAYER_NAME",
    "GOAT_INDEX_FINAL",
    "GOAT_INDEX",
    "CHAMPIONSHIPS_NORM"
]].copy()

ranking_final


,PLAYER_NAME,GOAT_INDEX_FINAL,GOAT_INDEX,CHAMPIONSHIPS_NORM
0,Michael Jordan,0.719585,0.688427,1.000000
1,Michael Jordan,0.707355,0.674839,1.000000
2,Michael Jordan,0.684943,0.649937,1.000000
3,LeBron James,0.670794,0.671252,0.666667
4,Michael Jordan,0.642925,0.603250,1.000000
5,LeBron James,0.639066,0.635999,0.666667
6,LeBron James,0.632457,0.628656,0.666667
7,LeBron James,0.631752,0.627872,0.666667
8,LeBron James,0.630088,0.626023,0.666667
9,Michael Jordan,0.627367,0.585963,1.000000


In [25]:
ranking_final = ranking_final.round(3)
ranking_final


,PLAYER_NAME,GOAT_INDEX_FINAL,GOAT_INDEX,CHAMPIONSHIPS_NORM
0,Michael Jordan,0.720,0.688,1.000
1,Michael Jordan,0.707,0.675,1.000
2,Michael Jordan,0.685,0.650,1.000
3,LeBron James,0.671,0.671,0.667
4,Michael Jordan,0.643,0.603,1.000
5,LeBron James,0.639,0.636,0.667
6,LeBron James,0.632,0.629,0.667
7,LeBron James,0.632,0.628,0.667
8,LeBron James,0.630,0.626,0.667
9,Michael Jordan,0.627,0.586,1.000


In [26]:
goat_career_final = (
    goat_final
    .groupby("PLAYER_NAME", as_index=False)
    .mean(numeric_only=True)
)


In [27]:
goat_career_final = goat_career_final.sort_values(
    "GOAT_INDEX_FINAL",
    ascending=False
)


In [28]:
ranking_final = goat_career_final[[
    "PLAYER_NAME",
    "GOAT_INDEX_FINAL",
    "GOAT_INDEX",
    "CHAMPIONSHIPS_NORM"
]].round(3)

ranking_final


,PLAYER_NAME,GOAT_INDEX_FINAL,GOAT_INDEX,CHAMPIONSHIPS_NORM
2,Michael Jordan,0.581,0.534,1.000
1,LeBron James,0.562,0.551,0.667
0,Kobe Bryant,0.435,0.391,0.833


In [29]:
ranking_final = ranking_final.reset_index(drop=True)
ranking_final.index += 1
ranking_final.index.name = "RANK"
ranking_final

,PLAYER_NAME,GOAT_INDEX_FINAL,GOAT_INDEX,CHAMPIONSHIPS_NORM
RANK,,,,
1,Michael Jordan,0.581,0.534,1.000
2,LeBron James,0.562,0.551,0.667
3,Kobe Bryant,0.435,0.391,0.833
